# 公募基金重仓股季度持股变化

计算以公募基金为主体，重仓股的季度变化

In [144]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from WindPy import w
import datetime
%matplotlib inline

In [145]:
w.start()

.ErrorCode=0
.Data=[Already conntected!]

In [146]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.0.185.138;DATABASE=JYDB;UID=js_dev;PWD=js_dev')

In [147]:
query = 'SELECT fdo.InvestAdvisorAbbrName, t.SecuAbbr, s.SecuCode, s.SecuAbbr, f.ReportDate, f.SharesHolding, f.MarketValue \
FROM MF_KeyStockPortfolio as f, SecuMain as t, SecuMain as s, MF_FundArchives as fd, MF_InvestAdvisorOutline as fdo \
WHERE f.InnerCode = t.InnerCode \
    AND f.StockInnerCode = s.InnerCode \
    AND fd.InnerCode = f.InnerCode \
    AND fd.InvestAdvisorCode = fdo.InvestAdvisorCode \
    ORDER BY f.ReportDate DESC, f.SharesHolding DESC'

In [148]:
df = pd.read_sql_query(query, con=conn)

In [149]:
df.head()

,InvestAdvisorAbbrName,SecuAbbr,SecuCode,SecuAbbr,ReportDate,SharesHolding,MarketValue
0,东证资管,东方红目标,601939,建设银行,2018-08-17,2200000.0,14344000.00
1,东证资管,东方红目标,002470,金 正 大,2018-08-17,1009800.0,6694974.00
2,东证资管,东方红目标,601668,中国建筑,2018-08-17,1000000.0,5400000.00
3,东证资管,东方红目标,002078,太阳纸业,2018-08-17,808682.0,6647366.04
4,东证资管,东方红目标,601166,兴业银行,2018-08-17,400000.0,5872000.00


In [150]:
df.columns = ['InvestAdvisorAbbrName', 'FundAbbr', 'SecuCode', 'SecuAbbr', 'ReportDate', 'SharesHolding', 'MarketValue']

In [151]:
for name in df['InvestAdvisorAbbrName'].unique():
    print name

东证资管
华夏
平安大华
富国
汇添富
博时
南方
安信
华安
银华
招商
嘉实
汇丰晋信
中欧
鹏华
易方达
上投摩根
兴全
华泰柏瑞
光大保德信
中邮创业
新华
工银瑞信
国泰
景顺长城
九泰
融通
广发
申万菱信
华商
万家
前海开源
国海富兰克林
诺安
财通
交银施罗德
大成
中银
中信保诚
中融
鹏扬
国投瑞银
国联安
东方
富安达
泓德
长信
长盛
中银国际证券
华宝
民生加银
建信
海富通
天弘
农银汇理
宝盈
泰达宏利
诺德
泰康资产
东方阿尔法
东海
金鹰
浦银安盛
信达澳银
圆信永丰
创金合信
中海
德邦
银河
长城
永赢
金元顺安
东吴
摩根士丹利华鑫
国寿安保
方正富邦
华富
华润元大
天治
泰信
汇安
浙商
益民
红土创新
英大
长安
兴业
金信
太平
西部利得
前海联合
中金
中加
华泰保兴
上银
北信瑞丰
兴银
国金
浙商资管
富荣
华融证券
鑫元
中科沃土
先锋
格林
人保资产
江信
华泰资管
红塔红土
东兴证券
山西证券
财通资管
中信建投
恒生前海
国都证券
中航
长江资管
国开泰富
嘉合
南华
渤海汇金
新沃
华宸未来


In [176]:
comp = df[df['InvestAdvisorAbbrName'] == u'海富通']

In [177]:
comp_df = comp.groupby(['ReportDate', 'SecuAbbr', 'SecuCode']).sum()

In [178]:
comp_df = comp_df.reset_index(level=[1]).reset_index(level=[1])

In [179]:
comp_df['Month'] = comp_df.index.map(lambda x: x.month)
comp_df['Day'] = comp_df.index.map(lambda x: x.day)

In [180]:
comp_df = comp_df[(comp_df['Month'].isin([3,6,9,12])) & (comp_df['Day'].isin([30,31]))]

In [181]:
comp_df.head()

,SecuCode,SecuAbbr,SharesHolding,MarketValue,Month,Day
ReportDate,,,,,,
2003-12-31,600205,S 山东铝,3513297.0,62677218.48,12,31
2003-12-31,600309,万华化学,3654843.0,49815510.09,12,31
2003-12-31,600104,上汽集团,3313992.0,45633669.84,12,31
2003-12-31,990018,上港集箱,3749757.0,53246549.40,12,31
2003-12-31,000063,中兴通讯,2929420.0,54780154.00,12,31


In [182]:
comp_df['ReportDate'] = comp_df.index

In [183]:
dates = comp_df.index.drop_duplicates().sort_values(ascending=False).values

In [184]:
for next_date, prev_date in zip(dates, dates[1:]):
    next_date_hold = comp_df[comp_df.index == next_date]['SecuAbbr']
    prev_date_hold = comp_df[comp_df.index == prev_date]['SecuAbbr']
    codes = prev_date_hold.loc[~prev_date_hold.isin(next_date_hold)].values
    add_df = comp_df[comp_df['SecuAbbr'].isin(codes)].copy()
    add_df = add_df[add_df.index == prev_date]
    add_df['MarketValue'] = 0
    add_df['SharesHolding'] = 0
    add_df['ReportDate'] = next_date
    comp_df = comp_df.append(add_df)

In [188]:
comp_df = comp_df.sort_values('ReportDate', ascending=False)

In [189]:
comp_df['Diff'] = comp_df['MarketValue'] - comp_df.groupby('SecuAbbr')['MarketValue'].shift(-1)

In [190]:
comp_df['Diff'] = comp_df['Diff'].fillna(comp_df['MarketValue'])

In [193]:
comp_df['SDiff'] = comp_df['SharesHolding'] - comp_df.groupby('SecuAbbr')['SharesHolding'].shift(-1)

In [194]:
comp_df['SDiff'] = comp_df['SDiff'].fillna(comp_df['SharesHolding'])

In [195]:
comp_df = comp_df.sort_values(['ReportDate', 'SDiff'], ascending=False)

In [196]:
comp_df = comp_df.set_index('ReportDate')

In [197]:
comp_df = comp_df[comp_df.index == comp_df.index[0]]

In [198]:
comp_df.shape

(136, 8)

In [199]:
comp_df.dropna().shape

(136, 8)

In [174]:
comp_df

,SecuCode,SecuAbbr,SharesHolding,MarketValue,Month,Day,Diff,SDiff
ReportDate,,,,,,,,
2018-06-30,002371,北方华创,3896800.0,1.935541e+08,6,30,1.935541e+08,3896800.0
2018-06-30,002410,广 联 达,6044027.0,1.676009e+08,6,30,1.676009e+08,NaN
2018-06-30,603986,兆易创新,1484407.0,1.610878e+08,6,30,1.610878e+08,NaN
2018-06-30,600519,贵州茅台,253493.0,1.854200e+08,6,30,1.483480e+08,199264.0
2018-06-30,603019,中科曙光,2817606.0,1.292436e+08,6,30,1.292436e+08,NaN
2018-06-30,600887,伊利股份,5067022.0,1.413699e+08,6,30,1.201117e+08,4320858.0
2018-06-30,300604,长川科技,2683870.0,9.761235e+07,6,30,9.761235e+07,NaN
2018-06-30,600271,航天信息,3771682.0,9.531040e+07,6,30,9.531040e+07,3771682.0
2018-06-30,300373,扬杰科技,3052562.0,8.730327e+07,6,30,8.730327e+07,NaN


In [138]:
def wind_code(x):
    if x.startswith('60'):
        return x + '.SH'
    else:
        return x + '.SZ'

In [139]:
data = w.wss('603345.SH', 'industry_citic', 'tradeDate=20180807;industryType=4')

In [140]:
print data.Data[0][0]

食品饮料-食品-肉制品


In [141]:
today = datetime.datetime.today()
codes = comp_df['SecuCode'].apply(lambda x: wind_code(x)).tolist()
param = 'tradeDate=%s;industryType=4'%(today.strftime('%Y%m%d'))
data = w.wss(codes, 'industry_citic', param)

In [142]:
comp_df['industry'] = data.Data[0]

In [143]:
comp_df

,SecuCode,SecuAbbr,SharesHolding,MarketValue,Month,Day,Diff,industry
ReportDate,,,,,,,,
2018-06-30,600276,恒瑞医药,14942398.0,1.132036e+09,6,30,8.699724e+08,医药-化学制药-化学制剂
2018-06-30,600763,通策医疗,23517012.0,1.144573e+09,6,30,4.909832e+08,医药-其他医药医疗-医疗服务
2018-06-30,600062,华润双鹤,28907540.0,7.096801e+08,6,30,4.546656e+08,医药-化学制药-化学制剂
2018-06-30,600271,航天信息,22937108.0,5.796207e+08,6,30,3.289814e+08,计算机-IT服务-系统集成及IT咨询
2018-06-30,000538,云南白药,4156651.0,4.445954e+08,6,30,2.978791e+08,医药-中药生产-中成药
2018-06-30,002032,苏 泊 尔,8567830.0,4.412432e+08,6,30,2.956405e+08,家电-小家电Ⅱ-小家电Ⅲ
2018-06-30,300037,新 宙 邦,10942350.0,2.931456e+08,6,30,2.710899e+08,基础化工-化学原料-其他化学原料
2018-06-30,002120,韵达股份,5792553.0,2.933236e+08,6,30,2.699165e+08,交通运输-公交物流-物流
2018-06-30,002024,苏宁易购,18250600.0,2.569684e+08,6,30,2.541010e+08,商贸零售-零售-连锁
